# Движок Tantivity для организации локальной поисковой системы

[Документация для Питона](https://tantivy-py.readthedocs.io/en/latest/)  
[Статья на Хабре](https://habr.com/ru/companies/otus/articles/959244/) с описанием возможностей движка.  


In [2]:
!pip install tantivy --break-system-packages

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 4.7 MB/s eta 0:00:00m eta 0:00:010:01:01


In [3]:
import tantivy

In [6]:
# Declaring our schema.
schema_builder = tantivy.SchemaBuilder()
schema_builder.add_text_field("title", stored=True)
schema_builder.add_text_field("body", stored=True)
schema_builder.add_integer_field("doc_id",stored=True)
schema = schema_builder.build()

# Creating our index (in memory)
index = tantivy.Index(schema)

# Creating our index (on drive)
persistent_index = tantivy.Index(schema, path="tantivity_index")

In [24]:
with open("lenta2018.txt", "rt") as news_file:
    news_text = news_file.read()

news_list = news_text.split("=====\n")
news_headers = [news.split("-----\n")[0] for news in news_list if len(news)>1]
# news_bodies = [len(news) for news in news_list if len(news)<10]
news_bodies = [news.split("-----\n")[1] for news in news_list if len(news)>1]

del news_text
del news_list

In [27]:
schema_builder_tok = tantivy.SchemaBuilder()
schema_builder_tok.add_text_field("body",  stored=True,  tokenizer_name='en_stem')
# schema_builder_tok.add_text_field("body",  stored=True,  tokenizer_name='ru_stem')

In [31]:
%%time
# Создаем индекс в оперативной памяти.
writer = index.writer()
counter = 1
for header, body in zip(news_headers, news_bodies):
    writer.add_document(tantivy.Document(
        doc_id=counter,
        title=[header],
        body=[body],
    ))
    counter += 1
# ... and committing
writer.commit()
writer.wait_merging_threads()
# Note that wait_merging_threads() must come at the end, 
# because the writer object will not be usable after this call.

CPU times: user 676 ms, sys: 35.7 ms, total: 712 ms
Wall time: 356 ms


In [32]:
%%time
# Создаем индекс на диске.
writer = persistent_index.writer()
counter = 1
for header, body in zip(news_headers, news_bodies):
    writer.add_document(tantivy.Document(
        doc_id=counter,
        title=[header],
        body=[body],
    ))
    counter += 1
# ... and committing
writer.commit()
writer.wait_merging_threads()
# Note that wait_merging_threads() must come at the end, 
# because the writer object will not be usable after this call.

CPU times: user 637 ms, sys: 54.2 ms, total: 691 ms
Wall time: 298 ms


Теперь займемся поиском.

In [33]:
# Reload the index to ensure it points to the last commit.
index.reload()
searcher = index.searcher()

In [42]:
%%time
query = index.parse_query("ученые исследовали", ["title", "body"])
(best_score, best_doc_address) = searcher.search(query, 3).hits[0]
best_doc = searcher.doc(best_doc_address)

CPU times: user 872 µs, sys: 33 µs, total: 905 µs
Wall time: 563 µs


In [35]:
best_doc

Document(body=[Американск],doc_id=[219],title=[Найдены де])

In [37]:
news_headers[218], news_bodies[218]

('Найдены десятки ранее неизвестных городов майя\n',
 'Американские археологи обнаружили в Гватемале следы более 60 тысяч ранее неизвестных сооружений, принадлежащих цивилизации майя. По мнению ученых, наличие многочисленных оросительных каналов, дорог, крепостей и пирамид указывает на существование в древности на исследованных территориях десятков крупных городов, сообщает ArsTechnica.Артефакты исчезнувшей цивилизации найдены на северо-востоке Гватемалы, всего при помощи лидаров (лазерных радаров) ученые исследовали более 2 тысяч квадратных километров местности.Материалы по теме00:05 — 28 февраля 2016ПонаехалиСуперматерик Сахул заселили раньше ЕвропыПолученные данные указывают, что цивилизация майя была гораздо более многочисленной, чем считалось ранее, а ее разрастание началось вокруг обнаруженных поселений. Согласно предварительным оценкам специалистов, на этой территории могли проживать порядка 10 миллионов человек.Древняя мезоамериканская цивилизация майя существовала на территори

In [40]:
best_doc["title"], best_doc["body"]

(['Найдены десятки ранее неизвестных городов майя\n'],
 ['Американские археологи обнаружили в Гватемале следы более 60 тысяч ранее неизвестных сооружений, принадлежащих цивилизации майя. По мнению ученых, наличие многочисленных оросительных каналов, дорог, крепостей и пирамид указывает на существование в древности на исследованных территориях десятков крупных городов, сообщает ArsTechnica.Артефакты исчезнувшей цивилизации найдены на северо-востоке Гватемалы, всего при помощи лидаров (лазерных радаров) ученые исследовали более 2 тысяч квадратных километров местности.Материалы по теме00:05 — 28 февраля 2016ПонаехалиСуперматерик Сахул заселили раньше ЕвропыПолученные данные указывают, что цивилизация майя была гораздо более многочисленной, чем считалось ранее, а ее разрастание началось вокруг обнаруженных поселений. Согласно предварительным оценкам специалистов, на этой территории могли проживать порядка 10 миллионов человек.Древняя мезоамериканская цивилизация майя существовала на террит

In [50]:
%%time
query = persistent_index.parse_query("ученые", ["title", "body"])
hits = searcher.search(query, 10).hits
# best_doc = searcher.doc(best_doc_address)

CPU times: user 761 µs, sys: 0 ns, total: 761 µs
Wall time: 494 µs


In [51]:
hits

[(10.973064422607422, <tantivy.tantivy.DocAddress at 0x700502b9f8d0>),
 (10.973064422607422, <tantivy.tantivy.DocAddress at 0x700502b9f830>),
 (10.973064422607422, <tantivy.tantivy.DocAddress at 0x700502b9ef10>),
 (10.464849472045898, <tantivy.tantivy.DocAddress at 0x700502b9fa30>),
 (10.464849472045898, <tantivy.tantivy.DocAddress at 0x700502b9f990>),
 (10.464849472045898, <tantivy.tantivy.DocAddress at 0x700502b9fef0>),
 (10.434577941894531, <tantivy.tantivy.DocAddress at 0x700502b9ff10>),
 (10.434577941894531, <tantivy.tantivy.DocAddress at 0x700502b9ff30>),
 (10.434577941894531, <tantivy.tantivy.DocAddress at 0x700502b9ff50>),
 (9.839756965637207, <tantivy.tantivy.DocAddress at 0x700502b9ff70>)]

In [52]:
for hit in hits:
    doc = searcher.doc(hit[1])
    print(hit)
    print(f"{doc['doc_id']}: {doc['title']}")

(10.973064422607422, <tantivy.tantivy.DocAddress object at 0x700502b9f8d0>)
[1110]: ['Ученые разрешили есть снег\n']
(10.973064422607422, <tantivy.tantivy.DocAddress object at 0x700502b9f830>)
[1110]: ['Ученые разрешили есть снег\n']
(10.973064422607422, <tantivy.tantivy.DocAddress object at 0x700502b9ef10>)
[1110]: ['Ученые разрешили есть снег\n']
(10.464849472045898, <tantivy.tantivy.DocAddress object at 0x700502b9fa30>)
[152]: ['Ученые предупредили о грядущей катастрофе\n']
(10.464849472045898, <tantivy.tantivy.DocAddress object at 0x700502b9f990>)
[152]: ['Ученые предупредили о грядущей катастрофе\n']
(10.464849472045898, <tantivy.tantivy.DocAddress object at 0x700502b9fef0>)
[152]: ['Ученые предупредили о грядущей катастрофе\n']
(10.434577941894531, <tantivy.tantivy.DocAddress object at 0x700502b9ff10>)
[169]: ['Ученые побороли неизлечимый рак\n']
(10.434577941894531, <tantivy.tantivy.DocAddress object at 0x700502b9ff30>)
[169]: ['Ученые побороли неизлечимый рак\n']
(10.4345779418

In [53]:
dir(hit[1])

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'doc',
 'segment_ord']

In [61]:
hits[0][1].doc, hits[0][1].segment_ord, hits[1][1].doc, hits[1][1].segment_ord, hits[2][1].doc, hits[2][1].segment_ord

(1332, 0, 1132, 1, 1405, 2)

Сниппеты

In [93]:
from tantivy import SnippetGenerator

query = persistent_index.parse_query("ученые", ["title", "body"])
hits = searcher.search(query, 10).hits

best_doc = searcher.doc(hits[3][1])
snippet_generator = SnippetGenerator.create(
    searcher, query, schema, "body"
)
snippet = snippet_generator.snippet_from_doc(best_doc)

In [94]:
highlights = snippet.highlighted()
first_highlight = highlights[0]
print(first_highlight.start, first_highlight.end)
doc_body = best_doc["body"][0]
doc_body = doc_body[:first_highlight.start] + '__' + doc_body[first_highlight.start: first_highlight.end] + '__'  + doc_body[first_highlight.end:]
print(doc_body)

0 12
__Ученые заяви__ли о возможном скором перевороте магнитных полюсов Земли, а также последующих за этим катаклизмах, сообщает The Daily Mail.Геофизик Дэниел Бейкер (Daniel Baker) из Колорадского университета в Боулдере (США) изложил свою гипотезу в журнале Undark Magazine. По его мнению, за последние 200 лет магнитное поле планеты ослабло на 15 процентов, что может быть признаком скорой геомагнитной инверсии. В таком случае некоторые районы Земли, отмечает ученый, окажутся практически полностью непригодными для жизни.Материалы по теме00:05 —  6 февраля 2017Это конецЗемле предрекли переворот магнитного поляГеофизик Ричард Холме (Richard Holme) из Ливерпульского университета (Великобритания) заявил, что переворот полюсов спровоцирует массовые сбои в работе электрических систем по всей планете.Инверсия полюсов, как полагают ученые, осуществляется в течение нескольких тысяч лет и приводит к кратковременному резкому ослаблению магнитного поля Земли. В результате вредное для живых организ

In [87]:
len(highlights)

1